In [1]:
import numpy as np
import pandas as pd

In [5]:
import scipy
from statsmodels.stats.weightstats import *

In [2]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')

In [3]:
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [4]:
data.shape

(72, 15750)

In [8]:
genes = data.columns.values[2:]

Часть 1: применение t-критерия Стьюдента

В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

для групп normal (control) и early neoplasia (treatment)
для групп early neoplasia (control) и cancer (treatment)
В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.



In [14]:
normal_vs_early_neoplasia = []
early_neoplasia_vs_cancer = []

In [15]:
for gene in genes:
    normal_vs_early_neoplasia.append(
        scipy.stats.ttest_ind(data[data.Diagnosis == 'normal'][gene], 
                          data[data.Diagnosis == 'early neoplasia'][gene], 
                          equal_var = False).pvalue)
    early_neoplasia_vs_cancer.append(
        scipy.stats.ttest_ind(data[data.Diagnosis == 'early neoplasia'][gene], 
                          data[data.Diagnosis == 'cancer'][gene], 
                          equal_var = False).pvalue)
    

In [21]:
sum([x < 0.05 for x in normal_vs_early_neoplasia])

1575

In [24]:
sum([x < 0.05 for x in early_neoplasia_vs_cancer])

3490

Часть 2: поправка методом Холма

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. 

Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5.

In [69]:
import statsmodels.stats.multitest as smm

In [70]:
reject, p_corrected, a1, a2 = smm.multipletests(normal_vs_early_neoplasia, 
                                            alpha = 0.025, 
                                            method = 'holm') 

In [71]:
indeces = [x < 0.025 for x in p_corrected]

In [72]:
fold_change = []
for ind, flag in enumerate(indeces):
    if flag:
        T = np.mean(data[data.Diagnosis == 'early neoplasia'][genes[ind]])
        C = np.mean(data[data.Diagnosis == 'normal'][genes[ind]])
        if T > C:
            fold_change.append(T/C)
        if T < C:
            fold_change.append(-C/T)
sum(list(map(lambda x: abs(x) > 1.5, fold_change)))

2

In [73]:
reject, p_corrected, a1, a2 = smm.multipletests(early_neoplasia_vs_cancer, 
                                            alpha = 0.025, 
                                            method = 'holm') 

In [74]:
indeces = [x < 0.025 for x in p_corrected]

In [75]:
sum(indeces)

79

In [76]:
fold_change = []
for ind, flag in enumerate(indeces):
    if flag:
        T = np.mean(data[data.Diagnosis == 'cancer'][genes[ind]])
        C = np.mean(data[data.Diagnosis == 'early neoplasia'][genes[ind]])
        if T > C:
            fold_change.append(T/C)
        if T < C:
            fold_change.append(-C/T)
sum(list(map(lambda x: abs(x) > 1.5, fold_change)))

77

In [77]:
reject, p_corrected, a1, a2 = smm.multipletests(normal_vs_early_neoplasia, 
                                            alpha = 0.025, 
                                            method = 'fdr_bh') 

In [78]:
indeces = [x < 0.025 for x in p_corrected]

In [79]:
sum(indeces)

4

In [80]:
fold_change = []
for ind, flag in enumerate(indeces):
    if flag:
        T = np.mean(data[data.Diagnosis == 'early neoplasia'][genes[ind]])
        C = np.mean(data[data.Diagnosis == 'normal'][genes[ind]])
        if T > C:
            fold_change.append(T/C)
        if T < C:
            fold_change.append(-C/T)
sum(list(map(lambda x: abs(x) > 1.5, fold_change)))

4

In [81]:
reject, p_corrected, a1, a2 = smm.multipletests(early_neoplasia_vs_cancer, 
                                            alpha = 0.025, 
                                            method = 'fdr_bh') 

In [82]:
indeces = [x < 0.025 for x in p_corrected]

In [83]:
sum(indeces)

832

In [84]:
fold_change = []
for ind, flag in enumerate(indeces):
    if flag:
        T = np.mean(data[data.Diagnosis == 'cancer'][genes[ind]])
        C = np.mean(data[data.Diagnosis == 'early neoplasia'][genes[ind]])
        if T > C:
            fold_change.append(T/C)
        if T < C:
            fold_change.append(-C/T)
sum(list(map(lambda x: abs(x) > 1.5, fold_change)))

524